In [59]:
import pandas as pd
import boto3
from botocore.config import Config
import os
from faker import Faker
from dotenv import load_dotenv
load_dotenv()
from datetime import datetime
import numpy as np
my_config = Config(
    region_name = 'us-west-1',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)
client = boto3.client(
    'dynamodb',
    aws_access_key_id=os.getenv('awsAccessKeyId'),
    aws_secret_access_key=os.getenv('awsSecretAccessKey'),
    config=my_config)
dynamodb = boto3.resource(
    'dynamodb',
    aws_access_key_id=os.getenv('awsAccessKeyId'),
    aws_secret_access_key=os.getenv('awsSecretAccessKey'),
    config=my_config)
ddb_exceptions = client.exceptions
fake = Faker(['it_IT', 'en_US', 'ja_JP','zh_CN','zh_TW'])

In [60]:
userlimited= pd.read_excel(r'C:\Users\shush\Documents\GitHub\bhpmJS\tools\final_list.xlsx')

In [61]:
userlimited

,email,limit
0,963236546@qq.com,Unlimited
1,metroantique@gmail.com,Unlimited
2,Jane04222003@hotmail.com,Unlimited
3,mgzhong168@gmail.com,30000
4,shushengacademy001@gmail.com,15000
5,yangbn2008@gmail.com,200000


In [62]:
userlimited.shape

(6, 2)

In [63]:
for i in range(userlimited.shape[0]):
    print(userlimited.iloc[i])
    print(userlimited.iloc[i]['email'])
    print(userlimited.iloc[i]['limit'])
    print('----------------')

email    963236546@qq.com
limit           Unlimited
Name: 0, dtype: object
963236546@qq.com
Unlimited
----------------
email    metroantique@gmail.com
limit                 Unlimited
Name: 1, dtype: object
metroantique@gmail.com
Unlimited
----------------
email    Jane04222003@hotmail.com
limit                   Unlimited
Name: 2, dtype: object
Jane04222003@hotmail.com
Unlimited
----------------
email    mgzhong168@gmail.com
limit                   30000
Name: 3, dtype: object
mgzhong168@gmail.com
30000
----------------
email    shushengacademy001@gmail.com
limit                           15000
Name: 4, dtype: object
shushengacademy001@gmail.com
15000
----------------
email    yangbn2008@gmail.com
limit                  200000
Name: 5, dtype: object
yangbn2008@gmail.com
200000
----------------


In [64]:
UserInfoTable = dynamodb.Table('UserInfo-7jzw3fqlhrcuzaarveonxbtczq-prod')
AuctionUserLimitationTable=dynamodb.Table('AuctionUserLimitation-7jzw3fqlhrcuzaarveonxbtczq-prod')
AuctionUserNumberTable=dynamodb.Table('AuctionUserNumber-7jzw3fqlhrcuzaarveonxbtczq-prod')


In [65]:
userInfo=UserInfoTable.scan()
userInfo['Items']

[{'__typename': 'UserInfo',
  'updatedAt': '2022-05-09T17:51:35.000Z',
  'userName': '75a2d1d4-c5ae-41e0-8b0e-57e4002d1cfa',
  'createdAt': '2022-05-09T17:51:35.000Z',
  'id': '997d0b25-b09d-4bd8-92ba-71eebc928120',
  'email': 'yizhouz@hotmail.com',
  'phone': '6479298511'},
 {'__typename': 'UserInfo',
  'updatedAt': '2022-05-09T17:51:35.000Z',
  'userName': '36ecdce2-d36f-4c38-9843-9e8d6abd3183',
  'createdAt': '2022-05-09T17:51:35.000Z',
  'id': '11ae3778-c38f-475e-bca7-1854f55a0a4d',
  'email': 'yuejiayi0826@163.com',
  'phone': '+16479973519'},
 {'__typename': 'UserInfo',
  'updatedAt': '2022-05-12T15:50:40.792Z',
  'userName': '08e3273f-1c9c-4604-97cd-c63412307bc6',
  'createdAt': '2022-05-12T15:50:40.792Z',
  'id': '08e3273f-1c9c-4604-97cd-c63412307bc6',
  'email': '963236546@qq.com'},
 {'__typename': 'UserInfo',
  'updatedAt': '2022-05-09T17:51:35.000Z',
  'userName': '42994de1-3aeb-4ec9-9999-8a1ac0a7853f',
  'createdAt': '2022-05-09T17:51:35.000Z',
  'id': '4c305755-d38e-404c-9

In [66]:
for i in range(userlimited.shape[0]):
    # print(userlimited.iloc[i])
    # print(userlimited.iloc[i]['email'])
    # print(userlimited.iloc[i]['limit'])
    # print('----------------')
    for j in userInfo['Items']:
        if j['email'].strip().lower() == userlimited.iloc[i]['email'].strip().lower():
            print(f"Found user,user Email:{userlimited.iloc[i]['email']} and userName: {j['userName']}")
            createdAt=datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z")
            userNumberID=fake.uuid4()
            auctionUserLimitationId=fake.uuid4()
            AuctionUserNumberData={'__typename': 'AuctionUserNumber',
                            'id':userNumberID,
                            'auctionAuctionUserNumbersId':'fdfa3b15-1731-43fc-b23e-30c1707b954c',
                            'number':300+i,
                            'owner':j['userName'],
                            'createdAt':createdAt,
                            'updatedAt':createdAt
                            }
            response = AuctionUserNumberTable.put_item(Item=AuctionUserNumberData)
            if userlimited.iloc[i]['limit']=='Unlimited':
                limitStatus='Unlimited'
                maxUserBidPrice=100000000
            else:
                limitStatus='Limited'
                maxUserBidPrice=userlimited.iloc[i]['limit']


            AuctionUserLimitationData={'__typename': 'AuctionUserLimitation',
                'id':auctionUserLimitationId,
                'auctionAuctionUserLimitationsId':'fdfa3b15-1731-43fc-b23e-30c1707b954c',
                'limitStatus':limitStatus,
                'maxUserBidPrice':maxUserBidPrice,
                'owner':j['userName'],
                'createdAt':createdAt,
                'updatedAt':createdAt
                }
            response = AuctionUserLimitationTable.put_item(Item=AuctionUserLimitationData)


    

Found user,user Email:963236546@qq.com and userName: 08e3273f-1c9c-4604-97cd-c63412307bc6
Found user,user Email:metroantique@gmail.com and userName: f75559ac-9a2c-4f61-b25e-8765a2b392f8
Found user,user Email:Jane04222003@hotmail.com and userName: 452b1f8e-54a3-4bba-9175-274a68a8ca2f
Found user,user Email:mgzhong168@gmail.com and userName: 4aeaf19a-1eb4-4f59-b6f5-551f34e0a79a
Found user,user Email:shushengacademy001@gmail.com and userName: 3e9d5586-a6a1-4fc3-bec0-de9a3cde245e
Found user,user Email:yangbn2008@gmail.com and userName: eb0b06d0-28a8-4598-95db-8fe133400709
